# Deep convolutional generative adversarial networks (DCGANs)

## An artificial intelligence exercise pitting authentic versus synthetic imagery

This is an experiment for producing completely artificial images or shoes through a DCGAN, based on [the wonderful repo](https://github.com/CShorten/BballShoesDCGANs/blob/master/Kicks_DCGAN.ipynb) and [blog post](https://medium.com/@connorshorten300/generating-basketball-shoes-with-dcgans-6cd72d521c01) by Dr. Connor Shorten (Github: [@CShorten](https://github.com/CShorten)). Rather than use the shipped datasets like MNIST and CIFAR-10, Connor uses a custom dataset of basketball shoes, which I sneakily found [hiding on his profile page](https://github.com/CShorten/NIKE_vs_ADIDAS) in [training](https://github.com/CShorten/NIKE_vs_ADIDAS/tree/master/TRAIN) and [testing](https://github.com/CShorten/NIKE_vs_ADIDAS/tree/master/TEST) sets. 

His project uses a number of shoe images based on the same general style, with varying colors, which he trains a DCGAN on to produce custom designs. It's a really neat concept, and a much more practical example for grabbing real-world data that needs to be preprocessed and massaged. The source images also were larger and in color, as opposed to MNIST's (50000 28, 28, 1) shape. It wasn't at all hard.

So as a clever twist to get the bespoke dataset into a format that the DCGAN expects - a NumPy shape of (140, 45, 45, 3) - the images needed to be decomposed to their raw pixel values, and then persisted by storing them as an array in NumPy's uncompressed NPZ format. The 140 images Connor uses with both the testing and training sets combined come out to 1.6MB on disk.

I upscaled the demo [on Google Colab](https://colab.research.google.com/drive/12tPO5nwTgAtQHCWugo72CuZJOa1rLneD) to make use of cloud GPUs and make the machine learning run consist of many more epochs, hopefully to get both the DCGAN's generator and discriminator models to converge (or get close to it). I used Google Drive File Stream to access the .npz file on disk in my share space. 

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings

from keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, Activation, Conv2D, Conv2DTranspose, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import initializers

%matplotlib inline
warnings.simplefilter('ignore')

## Local image preprocessing

In [ ]:
shoes = []
DIR = './images'

for filename in os.listdir(DIR):
#     image = cv2.imread(os.path.join(DIR, filename), cv2.IMREAD_GRAYSCALE)
    image = cv2.imread(os.path.join(DIR, filename))
    image = cv2.resize(image, (45, 45))
    
    shoes.append(image)

shoes_raw_pixels = np.array(shoes)
np.savez('shoes.npz', shoes_raw_pixels)
print('shoes shape', shoes_raw_pixels.shape)

In [ ]:
''' UNCOMMENT THIS CELL *ONLY* WHEN RUNNING THIS NOTEBOOK ON GOOGLE COLAB '''
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# shoes_data = np.load('/content/drive/My Drive/data_science/shoes.npz')
# print(shoes_data['arr_0'].shape)

### Load the array & verify that it persisted with the correct dimensions

In [ ]:
shoes_data = np.load('shoes.npz')

plt.imshow(shoes_data['arr_0'][42, :, :, 0], cmap='nipy_spectral')
plt.show()

## OK! So far, so good - now let's build the GAN...

### Assemble the generator and discriminator models

In [ ]:
img_rows = 45
img_cols = 45
channels = 3
img_shape = (img_rows, img_cols, channels)
z_dim = 100
init = initializers.RandomNormal(mean=0.0, stddev=0.02)

def build_generator(img_shape, z_dim):
    model = Sequential()
    model.add(Dense(256*5*5, input_dim=z_dim, kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 5, 256)))
    
    model.add(Conv2DTranspose(512, kernel_size=3, strides=2))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(3, kernel_size=3, strides=2, activation='tanh'))
    
    z = Input(shape=(z_dim, ))
    img = model(z)
    
    return Model(inputs=z, outputs=img)

def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, kernel_initializer=init))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(64, kernel_size=3, strides=2))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(96, kernel_size=3, strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    img = Input(shape=img_shape)
    prediction = model(img)
    
    return Model(inputs=img, outputs=prediction)

### Compile the discriminator, freeze its layers, then combine the models for the DCGAN

In [ ]:
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

generator = build_generator(img_shape, z_dim)
z = Input(shape=(100, ))
img = generator(z)

discriminator.trainable = False
prediction = discriminator(img)

gan = Model(inputs=z, outputs=prediction)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

### Setup the machine learning method and backpropagate the gradient, updating the weights in the generator only

In [ ]:
losses = []
accuracies = []

def train(iterations, batch_size, sample_interval):
    # ------------------
    # PRE-PROCESSING
    # ------------------
    
    # use the first np.npz.files array as the training set of the raw pixel data
    X_train = shoes_data['arr_0']
    
    # scale the training data to the hyperbolic tangent range: [-1, 1]
    X_train = (X_train.astype('float32') / 127.5) - 1.0
    
    # ------------------
    # LABELS PREPARATION
    # ------------------
    labels_authentic = np.ones((batch_size, 1))
    labels_synthetic = np.zeros((batch_size, 1))
    
    for iteration in range(iterations):
        # produce a random batch of images from the training data
        index = np.random.randint(0, X_train.shape[0], batch_size)
        images_authentic = X_train[index]
        
        # produce a collection of fake images based on random noise
        z_noise = np.random.normal(0, 1, (batch_size, 100))
        images_synthetic = generator.predict(z_noise)

        # compute the discriminator model's loss
        d_loss_authentic = discriminator.train_on_batch(images_authentic, labels_authentic)
        d_loss_synthetic = discriminator.train_on_batch(images_synthetic, labels_synthetic)
        d_loss = 0.5 * np.add(d_loss_authentic, d_loss_synthetic)
        
        # produce a new crop of fake images based on random noise
        z_noise = np.random.normal(0, 1, (batch_size, 100))
        images_synthetic = generator.predict(z_noise)
        
        # compute the generator model's loss
        # based on the combined GAN network model
        # the discriminator's losses are frozen and  
        # backpropagate through to update the generator 
        # NOTE: the GAN is learning gradients based on random noise and real labels
        g_loss = gan.train_on_batch(z_noise, labels_authentic)
        
        if iteration % sample_interval == 0:
            print('> %d [D loss: %f accuracy: %.2f%%] [G loss: %f]' % (iteration, d_loss[0], 100*d_loss[1], g_loss))
            
            losses.append((d_loss[0], g_loss))
            accuracies.append(100*d_loss[1])
            sample_images(iteration)
            
def sample_images(iteration, rows=4, cols=4):
    # create images to challennge the discriminator
    z_noise = np.random.normal(0, 1, (rows*cols, z_dim))
    images_synthetic = generator.predict(z_noise)
    
    # rescale images back to the range [1, 1]
    images_synthetic = 0.5 * images_synthetic + 0.5
    fig, axes = plt.subplots(rows, cols, figsize=(4, 4), sharey=True, sharex=True)
    count = 0
    
    # plot images produced by the generator
    for i in range(rows):
        for j in range(cols):
            axes[i, j].imshow(images_synthetic[count, :, :, 0], cmap='gray')
            axes[i, j].axis('off')
            count += 1

## Good to go! Let's set hyperparameters and run this sucker! (You might want to grab a cup off coffee for this part...)

In [ ]:
iterations = 10000
batch_size = 128
sample_interval = 1000

start = time.time()
train(iterations, batch_size, sample_interval)
print('[INFO] completed training in {} seconds'.format(time.time() - start))

### How'd we do in the loss department?

In [ ]:
# plot the loss and accuracy metrics
plt.figure(figsize=(15, 7))
plt.plot(losses[0])
plt.plot(losses[1])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Discriminator', 'Generator'], loc='center right')
plt.show()